In [331]:
import time
import datetime
import numpy as np

# @to_do: do BFS using Customer > Peer > Provider order for trial() function
# @to_do: grpah component need to change to three sets to provide three kinds of neighbors
class Internet:
    def __init__(self, last_updated_time, AS_set=None, graph_component = None):
        if(isinstance(last_updated_time, datetime.datetime)):
            self._time = last_updated_time
        else:
            print("Error: Internet: invalid last_updated_time passed")
        self.AS_set = np.array([], dtype = object)
        self.AS_numbers = self.set_AS_set(AS_set)
        for key in graph_component:
            flag = True
            if key not in self.AS_numbers:
                flag = False
                print('Error: Internet: graph component does not match the AS numbers passed in ', key)
        if flag:
            for key in graph_component:
                for neighbors in graph_component[key]:
                    tmp = self.get_AS_by_number(key)
                    neighbor_tmp = self.get_AS_by_number(neighbors)
                    # one direction, can change to two direction
                    tmp.set_neighbors(np.array([neighbor_tmp], dtype = object))
                
    def set_AS_set(self, AS_set):
        AS_number = []
        for AS in AS_set:
            if AS.number not in AS_number:
                self.AS_set = np.append(self.AS_set, AS)
                AS_number.append(AS.number)
            else:
                print('Error: Internet: redundent AS number ', AS.number)
        return AS_number
    
    def get_AS_by_number(self, as_number):
        for AS in self.AS_set:
            if AS.number == as_number:
                return AS
        print("AS not in as set")
        return None
    
    def add_connection(self, AS1, AS2):
        if AS1.number not in self.AS_numbers:
            self.AS_numbers.append(AS1.number)
            self.AS_set = np.append(self.AS_set, AS1)
        if AS2.number not in self.AS_numbers:
            self.AS_numbers.append(AS2.number)
            self.AS_set = np.append(self.AS_set, AS2)
        # one direction, can change to two direction by add following
        # AS2.set_neighbors(np.array([AS1], dtype = object))
        AS1.set_neighbors(np.array([AS2], dtype = object))
    def trial(self, AS1, AS2):
        queue = np.array([AS1],dtype = object)
        trial = {}
        visited = {AS1.number: 1}
        while queue.size > 0:
            current = queue[-1]
            queue = queue[:-1]
            if current.number == AS2.number:
                result = [AS2.number]
                while AS2.number in trial:
                    AS2.number = trial[AS2.number]
                    result.insert(0, AS2.number)
                return result
            for neighbor in current.neighbors:
                if neighbor.number not in visited:
                    trial[neighbor.number] = current.number
                    queue = np.append(neighbor, queue)
                    visited[neighbor.number] = 1
        return []
    def distance(self, AS1, AS2):
        len_trial = len(self.trial(AS1, AS2))
        if len_trial > 1:
            return len_trial - 1;
        print("The input ASes are not connected, or inputs are the same")
        return 0
    
    def get_edges(self):
        edgename = []
        for AS in self.AS_set:
            for neighbor in AS.neighbors:
                if {AS.number, neighbor.number} not in edgename:
                    edgename.append({AS.number, neighbor.number})
        return edgename
    def get_vertices(self):
        vtx = []
        for AS in self.AS_set:
            if AS.number not in self.AS_set:
                vtx.append(AS.number)
        return vtx  
    def get_prefix_set(self):
        prefixes = np.array([], dtype = object)
        for AS in self.AS_set:
            for routers in AS.routers:
                for prefix in routers.prefix_set:
                    if prefix not in prefixes:
                        prefixes = np.append(prefix, prefixes)
        return prefixes

In [332]:
import numpy as np
geo_location_set = ['Asia', 'US', 'Europe', 'Australia', 'Africa','SouthernAmerica']
as_type_set = ['enterprise', 'private', 'non-profit']

# @to_do: keep three sets of neighbors, Customer, Peer, Provider
class AS:
    def __init__(self, as_type=None, geo_location=None, routers = None, number=None, neighbors=None):
        if as_type in as_type_set:
            self.as_type = as_type
        else:
            print("Error: in AS: as_type not match")
        if geo_location in geo_location_set:
            self.geo_location = geo_location
        else:
            print("Error: in AS: geo_location not match")
        if( number >= 0 and number <= 10000):
            self.number = number
        else:
            print("Error: in AS: as_number does not match")
        self.routers = np.array([], dtype = object)
        self.neighbors = np.array([], dtype = object)
        self.set_routers_set(routers)
        if neighbors != None:
            self.set_neighbors(neighbors)
    def set_routers_set(self, routers):
        for router in routers:
            self.routers = np.append(self.routers, router)
    def set_neighbors(self, neighbors):
        for neighbor in neighbors:
            if isinstance(neighbor, AS):
                if neighbor.number != self.number and neighbor not in self.neighbors:
                    self.neighbors = np.append(self.neighbors, neighbor)
            else:
               print("neighbor type not correct")

In [333]:
import numpy as np
import re
regex_IPprefix = re.compile('^([0-9]{1,3}\.){3}[0-9]{1,3}(\/([0-9]|[1-2][0-9]|3[0-2]))?$')

# @to do: when set the hosts, check if host IP address match the prefix after mask
class Prefix:
    def __init__(self, prefix=None, hosts=None, mask = None):
        self.hosts = np.array([], dtype = object)
        self.prefix = prefix
        self.set_hosts(hosts, prefix)
    def set_hosts(self, hosts, prefix):
        for host in hosts:
            self.hosts = np.append(self.hosts, host)
    def contains(self, IP_address):
        for i in range(len(self.hosts)):
            if (self.hosts[i].IP == IP_address):
                return True
        return False
    def subnet_size(self):
        return len(self.hosts)

In [334]:
import numpy as np
import re
regex_IPprefix = re.compile('^([0-9]{1,3}\.){3}[0-9]{1,3}(\/([0-9]|[1-2][0-9]|3[0-2]))?$')

class Routers:
    def __init__(self, prefix_set=None):
        self.prefix_set = np.array([], dtype = object)
        self.set_prefix_set(prefix_set)
    def set_prefix_set(self, prefix_set):
        for prefix in prefix_set:
            if(regex_IPprefix.match(prefix.prefix)):
                self.prefix_set = np.append(self.prefix_set, prefix)
            else:
                print("Error: Routers: router prefix does not fit IPV4 regex")

In [335]:
import numpy as np
import re
regex_DNS = re.compile('^(?![0-9]+$)(?!-)[a-zA-Z0-9-]{,63}(?<!-)$')
regex_IP = re.compile(r'^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$')

class Host:
    def __init__(self, SSL_version=None, DNS_name=None, IP=None):
        self.set_DNS_name(DNS_name)
        self.set_SSL_version(SSL_version)
        self.set_IP(IP)
    
    def set_IP(self, IP):
        if(regex_IP.match(IP)):
            self.IP = IP
        else:
            print("Error: Hosts: IP type does not match IPV4")
    def set_DNS_name(self, DNS_name):
        if(regex_DNS.match(DNS_name)):
            self.DNS_name = DNS_name
        else:
            print("Error: Hosts: DNS_name type does not match")
    def set_SSL_version(self, SSL_version):
        if(isinstance(SSL_version, float)):
            self.SSL_version = SSL_version
        else:
            print("Error: Hosts: SSL_version type does not match")

In [336]:
# this function is used to test the functionality of the data structure
# run main.py will do the test
host = Host(3.5, 'abc', '200.4.5.7')
AS_list = np.array([], dtype = object)
Host_list = np.array([], dtype = object)
for i in range(10):
    Host_list = np.append(Host_list ,Host(3 + i/10, 'abc', '200.4.5.7'))

a = Prefix('200.4.0.1/24',Host_list)

a = Prefix('200.4.0.1/24',Host_list)
a.contains('200.4.5.7')

prefix_list = np.array([], dtype = object)
for i in range(5):
    prefix_list = np.append(prefix_list, Prefix('200.4.0.1/24',Host_list))

b = Routers(prefix_list).prefix_set[0].hosts[0].IP

router_list = np.array([], dtype = object)
for i in range(6):
    router_list = np.append(router_list, Routers(prefix_list))

as_system1 = AS('enterprise','US',router_list, 1,)
as_system2 = AS('private','Asia',router_list, 2,)
as_system3 = AS('non-profit','Europe',router_list, 3,)
as_system4 = AS('non-profit','Europe',router_list, 4,)
as_list = np.array([as_system1, as_system2, as_system3, as_system4], dtype = object)
graph_component = {1:[2,3],
                   2:[1],
                   3:[1, 2],}

a = Internet(datetime.datetime.now(), as_list, graph_component)
a.add_connection(as_system2, as_system4)
print(as_system1.routers)

In [337]:
print(as_system2.neighbors)

[<__main__.AS object at 0x114b04b50> <__main__.AS object at 0x114b04ed0>]


In [338]:
as_system2.geo_location = "US"

In [339]:
a.add_connection(as_system4, as_system3)
a.get_edges()

[{1, 2}, {1, 3}, {2, 4}, {2, 3}, {3, 4}]

In [340]:
a.trial(as_system4, as_system1)

[4, 3, 1]